In [1]:
## Cargo las librerias
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
 

##Cargo las librerias de omdb
import omdb
import requests

##Cargo librerias de scraping web
import re 
from bs4 import BeautifulSoup
import requests

##Para archivos y directorios
import io
import os
import chardet

##Librerias de tiempo
import time
from datetime import datetime
import time

##Libreria del zip
from zipfile import ZipFile

#Con esto podemos echar un vistazo a código HTML directamente en la Notebook
from IPython.display import HTML, display

In [14]:
##Importo el data los links
with open('./download.csv', 'rb') as f:
    result = chardet.detect(f.read())  # se puede usar readline si el archivo es muy largo

# Cargar la data
link_movies = pd.read_csv('./download.csv', sep = ';',  encoding=result['encoding'])


print('link_movies: ',link_movies.shape[0])
print(link_movies.info())


link_movies:  3692
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3692 entries, 0 to 3691
Data columns (total 3 columns):
download_url    3692 non-null object
imdb_id         3692 non-null object
title           3692 non-null object
dtypes: object(3)
memory usage: 86.6+ KB
None


In [15]:
### Extraigo el nombre de la pelicula desde el subtitulo
directorio = './Subtitulos/ZIP/'

os.listdir(directorio)
nombre_sub = re.compile('^([^(]+)')
puntos = re.compile("[^\w\s']")

indice = 0
archivos = pd.DataFrame()

for i in os.listdir(directorio):
    nombre = nombre_sub.match(i)[0]
    nombre = puntos.sub(' ', nombre)
    nombre = nombre.strip().lower()
    nombre = {'nombre_archivo': i, 'title': nombre}
    nombre = pd.DataFrame(nombre, index= {indice})
    archivos = pd.concat([archivos, nombre])
    indice += 1
    

In [16]:
### Me quedo con el titulo y el id de imdb
link_movies2 = link_movies.loc[:, ['imdb_id', 'title']]
### A los titulos les saco los espacios en blanco y llevo todo a minuscula
link_movies2.title = [puntos.sub('', i.strip().lower()) for i in link_movies2.title]

In [17]:
## Joineo por titulo para traerme que me quede el nombre del archivo y el id de imdb
nombre_autom = pd.merge(left = archivos, right=link_movies2, how='inner', on='title')

In [18]:
nombre_autom.shape

(3317, 3)

In [19]:
### Abro los zip y extraigo los subtitulos .srt y los renombro con el imdb id
directorio_salida = './Subtitulos/SRT/'
errores = pd.DataFrame()

for i in range(len(nombre_autom.nombre_archivo)):
    try:
        ##Abro el zip del subtitulo 
        z = ZipFile(directorio+nombre_autom.iloc[i, 0], 'r')

        ##Dentro del zip encuentro el archivo con extension srt
        srt = [m for m in z.namelist() if re.findall(pattern='.srt', string = m)][0]

        ##Del zip extraigo solo el subtitulo y lo reescribo con el id de imdb
        for f in z.filelist:
            if f.filename == srt:
                f.filename = nombre_autom.iloc[i,2] +'.srt'
                z.extract(member = f, path = directorio_salida)
    except Exception as e:
        print("Error inesperado")
        print(e, "\n")
        errores = pd.concat([errores, pd.DataFrame(dict(nombre_autom.iloc[i, :]), index= {indice})])
        continue    

Error inesperado
list index out of range 

Error inesperado
list index out of range 

Error inesperado
list index out of range 

Error inesperado
list index out of range 

Error inesperado
list index out of range 

Error inesperado
list index out of range 

Error inesperado
list index out of range 

Error inesperado
list index out of range 

Error inesperado
list index out of range 

Error inesperado
list index out of range 

Error inesperado
list index out of range 

Error inesperado
list index out of range 

Error inesperado
list index out of range 

Error inesperado
list index out of range 

Error inesperado
list index out of range 

Error inesperado
list index out of range 

Error inesperado
list index out of range 

Error inesperado
list index out of range 

Error inesperado
list index out of range 

Error inesperado
list index out of range 

Error inesperado
list index out of range 

Error inesperado
list index out of range 

Error inesperado
list index out of range 

Error inesp

In [20]:
## Los errores son porque no son subtitulos .srt (27 no tienen otro subt)
print(errores.head(5))
errores.shape

        imdb_id                                   nombre_archivo  \
3558  tt0804461      death.sentence.(2007).eng.1cd.(6551985).zip   
3558  tt0264395               basic.(2003).eng.1cd.(6466613).zip   
3558  tt0186045       drowning.mona.(2000).eng.1cd.(3678605).zip   
3558  tt2302755  olympus.has.fallen.(2013).eng.1cd.(6919237).zip   
3558  tt1185266        stan.helsing.(2009).eng.1cd.(3602931).zip   

                   title  
3558      death sentence  
3558               basic  
3558       drowning mona  
3558  olympus has fallen  
3558        stan helsing  


(84, 3)

In [23]:
### Analizo los zip que no cruzaron
no_cruzan = pd.merge(left = archivos, right=link_movies2, how='left', on='title', indicator=True)
no_cruzan = no_cruzan.loc[no_cruzan['_merge'] == 'left_only',:]
no_cruzan.head()
no_cruzan.shape

(243, 4)